# analysis.final.b.step2

This script creates important tables and figures

In [1]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

incidence_vars = [
    'Total Mortality(low estimate)',
    'Total Mortality(high estimate)',
    'PM Mortality, All Cause (low)',
    'PM Mortality, All Cause (high)',
    'PM Infant Mortality',
    'Total O3 Mortality',
    'O3 Mortality (Short-term exposure)',
    'O3 Mortality (Long-term exposure)',
    'Total Asthma Symptoms',
    'PM Asthma Symptoms, Albuterol use',
    'O3 Asthma Symptoms, Chest Tightness',
    'O3 Asthma Symptoms, Cough',
    'O3 Asthma Symptoms, Shortness of Breath',
    'O3 Asthma Symptoms, Wheeze',
    'Total Incidence, Asthma',
    'PM Incidence, Asthma',
    'O3 Incidence, Asthma',
    'Total Incidence, Hay Fever/Rhinitis',
    'PM Incidence, Hay Fever/Rhinitis',
    'O3 Incidence, Hay Fever/Rhinitis',
    'Total ER Visits, Respiratory',
    'PM ER Visits, Respiratory',
    'O3 ER Visits, Respiratory',
    'Total Hospital Admits, All Respiratory',
    'PM Hospital Admits, All Respiratory',
    'O3 Hospital Admits, All Respiratory',
    'PM Nonfatal Heart Attacks',
    'PM Minor Restricted Activity Days',
    'PM Work Loss Days',
    'PM Incidence Lung Cancer',
    'PM HA Cardio Cerebro and Peripheral Vascular Disease',
    'PM HA Alzheimers Disease',
    'PM HA Parkinsons Disease',
    'PM Incidence Stroke',
    'PM Incidence Out of Hospital Cardiac Arrest',
    'PM ER visits All Cardiac Outcomes',
    'O3 ER Visits, Asthma',
    'O3 School Loss Days, All Cause'
]


In [3]:
pip install kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.0 MB/s eta 0:00:00


In [2]:
"""
Read in combined results
"""

# Script ======================================================================
results_dir0 = "/content/drive/MyDrive/gpDept-ResearchDept/LNG Air Pollution/LNG Health - COBRA project/Version 5 analysis"
agg_df_f0 = results_dir0 + "/b.finalData.results/b.finalData.01-03.combined_results.csv"
agg_df3 = pd.read_csv(agg_df_f0)
agg_df3['Project Status'] = pd.Categorical(agg_df3['Project Status'], categories=['Operating', 'Under Construction', 'Planned'])

## Project-level analysis

**Briefing Table 3.** Estimated single-year health impacts by LNG project

In [4]:
"""
Single-Year Health Impacts
Table export
-----------------
Sorted by project status first
Then sorted by project-level mortality or terminal-level mortality, split by project
"""

# Table configuration ========================================================
tbl_yr = 2023
f_out = 'b.finalData.briefing_table.project_level_results.xlsx'
save_to_xlsx = False
million_usd_unit = True
sort_by = 'Project mortality' # Options: Terminal mortality, Project mortality, Alphabetical, Alphabetical/Status

pivot_indices = ['Project', 'Project Status', 'Terminal']
colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
        '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)',
        'Total Incidence, Asthma',
          #  '$ Total Incidence, Asthma',
        'Total Asthma Symptoms',
          #  '$ Total Asthma Symptoms',
        'PM Work Loss Days', 'O3 School Loss Days, All Cause'
]

# colset0 = [i for i in agg_df3.columns if (('Total' in i) and '$' not in i)]
# colset0 = [i for i in agg_df3.columns if ('Total' in i)]

# Script =====================================================================

colsetf = pivot_indices + colset0

# Table scripts ==============================================================
# Filter agg_df3 down to just the rows and fields we need
tbl1_0 = agg_df3[(agg_df3['Analysis Year'] == tbl_yr)][colsetf]

# Create metadata DF for merging pivoted tables
add_meta = tbl1_0[pivot_indices].drop_duplicates()

# Sum by project and convert negatives to positive
tbl1_1 = -pd.pivot_table(tbl1_0[(['Project'] + colset0)],
               index=['Project'],
               aggfunc="sum")

if sort_by == 'Terminal mortality':
    # Create another table that is summed by terminal
    tbl1_2 = -pd.pivot_table(tbl1_0[(['Terminal'] + colset0)],
                  index=['Terminal'],
                            values = ['Total Mortality(high estimate)', '$ Total Health Benefits(high estimate)'],
                            aggfunc="sum")

    # Rename columns for legibility
    tbl1_2.rename(
        columns = {'Total Mortality(high estimate)': 'Terminal Total Mortality(high estimate)',
                  '$ Total Health Benefits(high estimate)': '$ Terminal Total Health Benefits(high estimate)'
                  }, inplace=True)

    # Merge the project-level data with the terminal-level sums for table sorting
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
              .merge(tbl1_2, left_on='Terminal', right_on=tbl1_2.index)
    )

    # Sort by project status first and then terminal total mortality (so that projects of the same terminal are grouped together)
    tbl1_3.sort_values(by=['Project Status', 'Terminal Total Mortality(high estimate)'], ascending=[True, False], inplace=True)

elif sort_by == 'Project mortality':
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
    )
    tbl1_3.sort_values(by=['Project Status', 'Total Mortality(high estimate)'], ascending=[True, False], inplace=True)

elif sort_by == 'Alphabetical':
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
    )
    tbl1_3.sort_values(by=['Project'], inplace=True)

elif sort_by == 'Alphabetical/Status':
    tbl1_3 = (tbl1_1
              .merge(add_meta, left_on=tbl1_1.index, right_on='Project')
    )
    tbl1_3.sort_values(by=['Project Status', 'Project'], inplace=True)

tbl1_4 = tbl1_3[(pivot_indices + colset0)]
tbl1_5 = tbl1_4.drop(columns=['Terminal'])

if million_usd_unit:
    usd_unit = [i for i in colset0 if ('$'  in i)]
    for i in usd_unit:
        tbl1_5[i] = tbl1_5[i] / 1000000
        tbl1_5.rename(columns={i: f'{i} (million USD)'}, inplace=True)

if save_to_xlsx:
    xlsx_out = results_dir0 + "/b.finalData.results/" + f_out
    tbl1_5.to_excel(xlsx_out, index=False)

tbl1_5.head(5)

,Project,Project Status,Total Mortality(high estimate),Total Mortality(low estimate),$ Total Health Benefits(high estimate) (million USD),$ Total Health Benefits(low estimate) (million USD),"Total Incidence, Asthma",Total Asthma Symptoms,PM Work Loss Days,"O3 School Loss Days, All Cause"
28,Sabine Pass LNG Phase I,Operating,15.978999,13.006942,257.029691,213.649877,81.952719,12620.063905,319.109334,7032.201060
29,Sabine Pass LNG Phase II,Operating,7.988900,6.502891,128.504474,106.814841,40.965768,6309.651328,159.554130,3515.722554
3,Cameron LNG Phase I,Operating,7.895032,5.593856,124.158864,90.571142,31.970255,5005.983746,245.433006,2495.456260
10,Cove Point LNG,Operating,7.771505,4.322651,117.901895,67.562791,18.081327,3023.066936,418.535316,962.822720
7,Corpus Christi LNG Stage I,Operating,7.738554,6.446611,128.005790,109.148722,51.960262,7881.293909,162.341922,4485.247807


**Briefing Figure 1.** Single-year mortality estimates by LNG terminal

In [5]:
"""
Project-level Health Impacts Bar Chart (high)
"""

import plotly.express as px
import plotly.graph_objects as go

color_map = {
    status: color for status, color in zip(tbl1_4['Project Status'].unique(), ['#003B4A', '#D54400', '#F7BE00'])
}


tbl1_4.sort_values(by=['Project Status'], inplace=True)

fig1 = px.bar(tbl1_4, y='Terminal', x="Total Mortality(high estimate)", color='Project Status', color_discrete_map=color_map)
fig1.update_layout(yaxis={'categoryorder':'total ascending'},

                   height=600, width=1000,
                                     legend=dict(
                            yanchor="bottom",
                            y=0.12,
                            xanchor="right",
                            x=0.92
                        ),
                   xaxis=dict(title='Premature Deaths (high estimate)')
                   )

# fig1.write_image(results_dir0 + "/b.finalData.results/project_impacts.svg", engine="kaleido")
# fig1.write_image(results_dir0 + "/b.finalData.results/project_impacts.pdf", engine="kaleido")
# fig1.write_image(results_dir0 + "/b.finalData.results/project_impacts.jpg")

fig1.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(required_grouper, sort=False)  # skip one_group groupers


In [ ]:
"""
Appendix: Project-level Health Impacts Bar Chart (low)
"""

import plotly.express as px
import plotly.graph_objects as go

color_map = {
    status: color for status, color in zip(tbl1_4['Project Status'].unique(), ['#003B4A', '#D54400', '#F7BE00'])
}


tbl1_4.sort_values(by=['Project Status'], inplace=True)

fig1 = px.bar(tbl1_4, y='Terminal', x="Total Mortality(low estimate)", color='Project Status', color_discrete_map=color_map)
fig1.update_layout(yaxis={'categoryorder':'total ascending'},

                   height=600, width=1000,
                                     legend=dict(
                            yanchor="bottom",
                            y=0.12,
                            xanchor="right",
                            x=0.92
                        ),
                                      xaxis=dict(title='Premature Deaths (low estimate)')

                   )

fig1.write_image(results_dir0 + "/b.finalData.results/project_impacts-low.svg", engine="kaleido")
fig1.write_image(results_dir0 + "/b.finalData.results/project_impacts-low.pdf", engine="kaleido")
fig1.write_image(results_dir0 + "/b.finalData.results/project_impacts-low.jpg")

fig1.show()

## County-level analysis

In [ ]:
"""
Impacts by county (Operating Projects only)
Table export
---------------
Sorted by county with the greatest total health impacts
Then, within the county, subtotals by project status
"""

# Table configuration ========================================================
tbl_yr = 2023
save_to_xlsx = False
f_out1 = 'b.finalData.briefing_table.county_level_results-sorted_by_total.xlsx'
f_out2 = 'b.finalData.briefing_table.county_level_results-sorted_by_percapita.xlsx'
million_usd_unit = True

pivot_indices = ['Destination County', 'Destination State', 'Project Status', 'Terminal']

# colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
#         'Total Mortality(high estimate) PER MILLION', 'Total Mortality(low estimate) PER MILLION',
#         'Delta PM 2.5', 'Delta O3',
#         '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)'
# ]
colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
        'Total Mortality(high estimate) PER MILLION', 'Total Mortality(low estimate) PER MILLION',
        '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)',
        'Total Incidence, Asthma',
          #  '$ Total Incidence, Asthma',
        'Total Asthma Symptoms',
           # '$ Total Asthma Symptoms',
        'PM Work Loss Days', 'O3 School Loss Days, All Cause'
]

colsetf = pivot_indices + colset0

# Table script ===============================================================

tbl2_0 = agg_df3[((agg_df3['Analysis Year'] == tbl_yr) & (agg_df3['Project Status'] == 'Operating'))][colsetf]
tbl2_0['County, State'] = tbl2_0[['Destination County', 'Destination State']].agg(', '.join, axis=1)
tbl2_0.drop(['Destination County', 'Destination State'], axis=1, inplace=True)

tbl2_1 = -pd.pivot_table(tbl2_0[(['County, State'] + colset0)],
               index=['County, State'],
               aggfunc="sum")[colset0]

# Add column for most impacting LNG terminal
tbl2_2 = -pd.pivot_table(tbl2_0[(['County, State', 'Terminal'] + colset0)],
               index=['County, State', 'Terminal'],
               aggfunc="sum")[colset0]
tbl2_2.reset_index(inplace=True)
tbl2_2.sort_values(by=['$ Total Health Benefits(high estimate)'], ascending=False, inplace=True)

def get_most_impacting_terminals(group):
    n_terminals = 3
    group_total = group['$ Total Health Benefits(high estimate)'].sum()
    str_all = []

    for i in np.arange(n_terminals):
        terminal_name = group.iloc[i]['Terminal']
        terminal_per = group.iloc[i]['$ Total Health Benefits(high estimate)']/group_total * 100
        str_out = f'{terminal_name} ({terminal_per:.0f}%)'
        str_out = str_out.replace(' LNG', '')
        str_all.append(str_out)

    return (', '.join(str_all))

max_terminal = tbl2_2.groupby('County, State').apply(get_most_impacting_terminals)
max_terminal.name = 'Most impacting terminals'

tbl2_1 = tbl2_1.merge(max_terminal, left_on = tbl2_1.index, right_on=max_terminal.index)
tbl2_1.rename(columns = {'key_0': 'County, State'}, inplace=True)

by_total = tbl2_1.sort_values(by=['Total Mortality(high estimate)'], ascending=False)
by_intenstity = tbl2_1.sort_values(by=['Total Mortality(high estimate) PER MILLION'], ascending=False)

if million_usd_unit:
    usd_unit = [i for i in colset0 if ('$'  in i)]
    for i in usd_unit:
        by_total[i] = by_total[i] / 1000000
        by_intenstity[i] = by_intenstity[i] / 1000000
        by_total.rename(columns={i: f'{i} (million USD)'}, inplace=True)
        by_intenstity.rename(columns={i: f'{i} (million USD)'}, inplace=True)

if save_to_xlsx:
  xlsx_out1 = results_dir0 + "/b.finalData.results/" + f_out1
  xlsx_out2 = results_dir0 + "/b.finalData.results/" + f_out2
  by_total.to_excel(xlsx_out1, index=False)
  # by_intenstity.to_excel(xlsx_out2, index=False)



,"County, State",Total Mortality(high estimate),Total Mortality(low estimate),Total Mortality(high estimate) PER MILLION,Total Mortality(low estimate) PER MILLION,$ Total Health Benefits(high estimate) (million USD),$ Total Health Benefits(low estimate) (million USD),"Total Incidence, Asthma",Total Asthma Symptoms,PM Work Loss Days,"O3 School Loss Days, All Cause",Most impacting terminals
1200,"Harris, Texas",4.565338e+00,2.794198e+00,0.894176,0.547277,7.470841e+01,4.885704e+01,3.085005e+01,4756.057135,2.958645e+02,2.163682e+03,"Sabine Pass (31%), Freeport (28%), Corpus Chri..."
335,"Calcasieu, Louisiana",2.729690e+00,1.911365e+00,13.132119,9.195284,4.244390e+01,3.049972e+01,9.406992e+00,1480.305692,7.335167e+01,7.201287e+02,"Sabine Pass (57%), Cameron (23%), Calcasieu Pa..."
196,"Bexar, Texas",1.034973e+00,7.691505e-01,0.484034,0.359715,1.729552e+01,1.341560e+01,7.653548e+00,1179.853429,3.695555e+01,6.412960e+02,"Corpus Christi (73%), Sabine Pass (16%), Camer..."
707,"Dallas, Texas",1.033979e+00,7.882707e-01,0.366265,0.279228,1.759281e+01,1.400648e+01,8.883622e+00,1337.230965,3.773814e+01,7.300362e+02,"Corpus Christi (48%), Sabine Pass (32%), Camer..."
1409,"Jefferson, Texas",8.171385e-01,5.181184e-01,3.186553,2.020480,1.263645e+01,8.271991e+00,2.647787e+00,423.801447,3.039968e+01,1.869612e+02,"Sabine Pass (53%), Cameron (22%), Calcasieu Pa..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2576,"Sherman, Oregon",2.847208e-08,1.957236e-08,0.000018,0.000012,4.317003e-07,3.018010e-07,5.658550e-08,0.000009,6.830644e-07,4.398147e-06,"Corpus Christi (44%), Sabine Pass (24%), Camer..."
51,"Alpine, California",2.226977e-08,1.548563e-08,0.000020,0.000014,3.529868e-07,2.539662e-07,9.495537e-08,0.000014,7.072556e-07,8.464698e-06,"Corpus Christi (42%), Sabine Pass (32%), Camer..."
2877,"Wahkiakum, Washington",1.788635e-08,9.441874e-09,0.000004,0.000002,2.634727e-07,1.402181e-07,9.879762e-09,0.000002,4.743375e-07,1.709780e-07,"Corpus Christi (34%), Sabine Pass (23%), Calca..."
267,"Bristol City, Virginia",-0.000000e+00,-0.000000e+00,-0.000000,-0.000000,3.305795e-05,3.305795e-05,-0.000000e+00,-0.000000,3.105241e-02,-0.000000e+00,"Sabine Pass (32%), Cove Point (18%), Calcasieu..."


In [ ]:
"""
Impacts by county by project status
Table export
---------------
Sorted by county with the greatest total health impacts
Then, within the county, subtotals by project status
"""

# Table configuration ========================================================
tbl_yr = 2023
save_to_xlsx = False

pivot_indices = ['Destination County', 'Destination State', 'Project Status', 'Terminal']
# colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
#         'Total Mortality(high estimate) PER MILLION', 'Total Mortality(low estimate) PER MILLION',
#         'Delta PM 2.5', 'Delta O3',
#         '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)'
# ]
colset0 = ['Total Mortality(high estimate)', 'Total Mortality(low estimate)',
        'Total Mortality(high estimate) PER MILLION', 'Total Mortality(low estimate) PER MILLION',
        '$ Total Health Benefits(high estimate)', '$ Total Health Benefits(low estimate)',
        'Total Incidence, Asthma', '$ Total Incidence, Asthma',
        'Total Asthma Symptoms', '$ Total Asthma Symptoms',
        'PM Work Loss Days', 'O3 School Loss Days, All Cause'
]
colsetf = pivot_indices + colset0

# Table script ===============================================================

tbl2_0 = agg_df3[(agg_df3['Analysis Year'] == tbl_yr)][colsetf]
tbl2_0['County, State'] = tbl2_0[['Destination County', 'Destination State']].agg(', '.join, axis=1)
tbl2_0.drop(['Destination County', 'Destination State'], axis=1, inplace=True)

tbl2_1 = -pd.pivot_table(tbl2_0,
               index=['County, State', 'Project Status'],
               aggfunc="sum")[colset0]
tbl2_sorter = -pd.pivot_table(tbl2_0[(['County, State'] + colset0)],
               index=['County, State'],
               aggfunc="sum")[colset0]

tbl2_sorter.rename(
    columns = {'Total Mortality(high estimate)': 'County Total Mortality(high estimate)',
               '$ Total Health Benefits(high estimate)': '$ County Total Health Benefits(high estimate)'
               }, inplace=True)

tbl2_sorter.sort_values(by=['$ County Total Health Benefits(high estimate)'], ascending=False, inplace=True)

tbl2_2 = tbl2_1.loc[tbl2_sorter.index]

if save_to_xlsx:
  xlsx_out = results_dir0 + "/b.finalData.results/b.finalData.01-03.combined_results.table2.xlsx"
  tbl2_2.to_excel(xlsx_out)

tbl2_2.head(5*3)

Total Mortality(high estimate)  \
County, State        Project Status                                       
Harris, Texas        Operating                                 4.565338   
                     Under Construction                        2.765266   
                     Planned                                   3.912136   
Calcasieu, Louisiana Operating                                 2.729690   
                     Under Construction                        1.607650   
                     Planned                                   2.924764   
Jefferson, Texas     Operating                                 0.817138   
                     Under Construction                        1.631527   
                     Planned                                   1.605720   
Jefferson, Louisiana Operating                                 0.283662   
                     Under Construction                        0.667575   
                     Planned                                   1.900061   
Orange, Texas        Operating                                 0.678149   
                     Under Construction                        0.931754   
                     Planned                                   1.055731   

                                         Total Mortality(low estimate)  \
County, State        Project Status                                      
Harris, Texas        Operating                                2.794198   
                     Under Construction                       1.522980   
                     Planned                                  2.061363   
Calcasieu, Louisiana Operating                                1.911365   
                     Under Construction                       0.983564   
                     Planned                                  1.690500   
Jefferson, Texas     Operating                                0.518118   
                     Under Construction                       0.924233   
                     Planned                                  0.873941   
Jefferson, Louisiana Operating                                0.221681   
                     Under Construction                       0.356227   
                     Planned                                  0.996673   
Orange, Texas        Operating                                0.457677   
                     Under Construction                       0.574678   
                     Planned                                  0.611817   

                                         Total Mortality(high estimate) PER MILLION  \
County, State        Project Status                                                   
Harris, Texas        Operating                                             0.894176   
                     Under Construction                                    0.541611   
                     Planned                                               0.766239   
Calcasieu, Louisiana Operating                                            13.132119   
                     Under Construction                                    7.734158   
                     Planned                                              14.070593   
Jefferson, Texas     Operating                                             3.186553   
                     Under Construction                                    6.362383   
                     Planned                                               6.261741   
Jefferson, Louisiana Operating                                             0.630257   
                     Under Construction                                    1.483258   
                     Planned                                               4.221668   
Orange, Texas        Operating                                             7.877463   
                     Under Construction                                   10.823374   
                     Planned                                              12.263512   

                          

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Assuming fig2_0 is already prepared as per your original code
n_counties = 25
fig2_0 = tbl2_2.loc[tbl2_sorter.index[0:n_counties]]
fig2_0.reset_index(inplace=True)
fig2_0.sort_values(by=['Project Status', "Total Mortality(high estimate)"], ascending=[True, False], inplace=True)

# Define a color map for each project status
color_map = {
    status: color for status, color in zip(fig2_0['Project Status'].unique(), px.colors.qualitative.Plotly)
}

# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)

# First bar chart
for status in fig2_0['Project Status'].unique():
    subset = fig2_0[fig2_0['Project Status'] == status]
    fig.add_trace(go.Bar(x=subset['County, State'], y=subset["Total Mortality(high estimate)"],
                         name=status, marker_color=color_map[status], showlegend=True), row=1, col=1)

# Second bar chart
for status in fig2_0['Project Status'].unique():
    subset = fig2_0[fig2_0['Project Status'] == status]
    fig.add_trace(go.Bar(x=subset['County, State'], y=subset["Total Mortality(high estimate) PER MILLION"],
                         name=status, marker_color=color_map[status], showlegend=False), row=2, col=1)

# Update layout for stacked bars
fig.update_layout(barmode='stack', height=800, width=1000, title_text="Impacts by County",
                  yaxis=dict(title='Total Mortality (high estimate)'),
                  yaxis2=dict(title='Total Mortality (high estimate) PER MILLION'),
                  xaxis=dict(title='County, State'))

fig.show()


## EJ analysis

In [ ]:
""" Functions for calculating differential exposure """

import pandas as pd
import numpy as np
import math

import progressbar

def calc_numerator(df, pollutant_col, race, same_state=False):
    num_sum = 0

    if same_state:
      df = df[df['State'] == df['Project State']]
    for ci in np.arange(len(df)):
      row = df.iloc[ci]
      county_prod = row[pollutant_col] * row['Total Population'] * row[race]/100

      if not math.isnan(county_prod):
        num_sum += county_prod

    return num_sum

def calc_denom(df, race, same_state=False):
    denom_sum = 0

    if same_state:
      df = df[df['State'] == df['Project State']]

    for ci in np.arange(len(df)):
      row = df.iloc[ci]
      county_prod = row['Total Population'] * row[race]/100
      if not math.isnan(county_prod):
        denom_sum += county_prod

    return denom_sum

def pop_weighted_exposure(df, pollutant_col, race, same_state=False):
  return (calc_numerator(df, pollutant_col, race, same_state)/calc_denom(df, race, same_state))

def calculate_population_weighted_exposure(df, projects, pollutant_cols, calc_groups, analysis_yr, scope):
    """
    Calculate population-weighted exposure by racial/ethnic group for each project and pollutant.

    Parameters:
    df1 (DataFrame): The input dataframe.
    projects (list): List of projects.
    pollutant_cols (list): List of pollutant columns.
    calc_groups (list): List of calculation groups.
    baseline (any): The baseline value to filter the dataframe.
    scope (str): national or state

    Returns:
    DataFrame: A dataframe containing population-weighted exposure by project, pollutant, and group.
    """

    col1 = []
    col2 = []
    col3 = []
    col4 = []

    if scope=='National':
        same_state = False
    else:
        same_state = True

    for i in progressbar.progressbar(range(len(projects))):
        for j in pollutant_cols:
            for k in calc_groups:
                df_i = df[(df['Analysis Year'] == analysis_yr) & (df['Project'] == projects[i])]

                col1.append(projects[i])
                col2.append(j)
                col3.append(k)
                col4.append(pop_weighted_exposure(df_i, j, k, same_state=same_state))

    exposure_df = pd.DataFrame({'Project': col1, 'Pollutant': col2, 'Group': col3, 'Exposure': col4})

    return exposure_df


In [ ]:
# Settings ==================================================================
analysis_yr = 2023
scopes = ['National', 'State']

# Read-in ACS data and give it better column names ==========================
f_acs = '/content/drive/MyDrive/gpDept-ResearchDept/LNG Air Pollution/LNG Health - COBRA project/Version 5 analysis/ACSDP5Y2022.DP05_2024-05-09T163456/ACSDP5Y2022.DP05-Data - FIPS Edit.csv'
acs_cols = {'FIPS_ACS': 'FIPS',
            'DP05_0066PE': 'pWhite', 'DP05_0066PM': 'pWhiteMarginError',
            'DP05_0067PE': 'pBlackAA', 'DP05_0067PM': 'pBlackAAMarginError',
            'DP05_0068PE': 'pAmerIndianAN', 'DP05_0068PM': 'pAmerIndianANMarginError',
            'DP05_0069PE': 'pAsian', 'DP05_0069PM': 'pAsianMarginError',
            'DP05_0070PE': 'pNativeHawaiianPI', 'DP05_0070PM': 'pNativeHawaiianPIMarginError',
            'DP05_0071PE': 'pOther', 'DP05_0071PM': 'pOtherMarginError',
            'DP05_0073PE': 'pHispanicLatino', 'DP05_0073PM': 'pHispanicLatinoMarginError' # Fill with NA ("*****" bc sum?)
            }
acs = pd.read_csv(f_acs, header=[0,1])

acs1 = acs[acs_cols.keys()]
acs1.columns = acs1.columns.droplevel(1)
acs1.rename(columns=acs_cols, inplace=True)

# Merge it with main dataframe ================================================
df0 = agg_df3[agg_df3['Analysis Year'] == analysis_yr]
df1 = (df0.merge(acs1, left_on='Destination FIPS', right_on='FIPS', how='left'))

# Connecticut issue... Counties to Planning Regions??
print(np.unique(df0['Destination FIPS'][~df0['Destination FIPS'].isin(acs1['FIPS'])]))

# Add state column to df
# To pick up tomorrow

# Create lists to loop through for calculating pop-weighted exposure ==========
projects = np.unique(df1['Project'])
calc_groups = [list(acs_cols.values())[i] for i in np.arange(1, 14, 2)]
pollutant_cols = ['Delta PM 2.5', 'Delta O3']
[i for i in df1.columns]

<ipython-input-45-bcb1991e89f6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs1.rename(columns=acs_cols, inplace=True)


[9001 9003 9005 9007 9009 9011 9013 9015]


['destindx',
 'Destination FIPS',
 'Destination State',
 'Destination County',
 'Base PM 2.5',
 'Control PM 2.5',
 'Delta PM 2.5',
 'Base O3',
 'Control O3',
 'Delta O3',
 '$ Total Health Benefits(low estimate)',
 '$ Total Health Benefits(high estimate)',
 'Total Mortality(low estimate)',
 '$ Total Mortality(low estimate)',
 'Total Mortality(high estimate)',
 '$ Total Mortality(high estimate)',
 'PM Mortality, All Cause (low)',
 '$ PM Mortality, All Cause (low)',
 'PM Mortality, All Cause (high)',
 '$ PM Mortality, All Cause (high)',
 'PM Infant Mortality',
 '$ PM Infant Mortality',
 'Total O3 Mortality',
 '$ Total O3 Mortality',
 'O3 Mortality (Short-term exposure)',
 '$ O3 Mortality (Short term exposure)',
 'O3 Mortality (Long-term exposure)',
 '$ O3 Mortality (Long-term exposure)',
 'Total Asthma Symptoms',
 '$ Total Asthma Symptoms',
 'PM Asthma Symptoms, Albuterol use',
 '$ PM Asthma Symptoms, Albuterol use',
 'O3 Asthma Symptoms, Chest Tightness',
 '$ O3 Asthma Symptoms, Chest Ti

In [ ]:
"""
Calculate population-weighted exposure by racial/ethnic group for each project and pollutant
"""

results = {}

for scope in scopes:
    results[scope] = (
        calculate_population_weighted_exposure(df1, projects[0:1], pollutant_cols, calc_groups, analysis_yr, scope)
    )

  0% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

State
True


KeyError: 'State'

In [ ]:
import matplotlib.pyplot as plt

f2 = '/content/drive/MyDrive/gpDept-ResearchDept/LNG Air Pollution/LNG Health - COBRA project/Version 5 analysis/240515-COBRA_Input_Reviewed_NoLegal-fix_CP_GP.xlsx'
pstatus = pd.read_excel(f2, sheet_name='Target List')[['Project', 'Project Status']]
# Note: Freeport might need custom working

df1 = (total_exposure_df.merge(
    instate_exposure_df, on=['Project', 'Pollutant', 'Group']
    , suffixes=('_national', '_instate'))
  .pivot(index=['Project', 'Group'], columns='Pollutant', values=['Exposure_national', 'Exposure_instate'])
  .reset_index(names=['Project', 'Group'])
)

df1.columns = ["_".join(a).strip('_') for a in df1.columns.to_flat_index()]

pt = pd.pivot(
    df1,
    values=['Exposure_national_DELTA_O3', 'Exposure_national_DELTA_PM',
            'Exposure_instate_DELTA_O3', 'Exposure_instate_DELTA_PM'],
    index='Project',
    columns = 'Group'
)

pt2 = pt.copy()

for top_level in pt2.columns.get_level_values(0).unique():
    # Get all columns under the current top-level column index
    columns = pt2[top_level].columns

    # Iterate over each racial group (except 'pWhite') to perform the division
    for group in columns:
          pt2[(top_level, group)] = (
              (pt2[(top_level, group)] - pt2[(top_level, 'pWhite')])
                / pt2[(top_level, 'pWhite')])

# Set the column names to include the ratio suffix for clarity
pt2.columns = pd.MultiIndex.from_tuples([(top, f'{group}_relative')
                                              for top, group in pt2.columns])
pt2.sort_values(by=('Exposure_national_DELTA_O3', 'pBlackAA_relative'), ascending=False, inplace=True)

pt2_op = pt2[pt2.index.isin(pstatus[pstatus['Project Status']=='Operating']['Project'])]
pt2_construction = pt2[pt2.index.isin(pstatus[pstatus['Project Status']=='Under Construction']['Project'])]
pt2_planned = pt2[pt2.index.isin(pstatus[pstatus['Project Status']=='Planned']['Project'])]

def create_plots(df, titles):
  # Assuming your dataframe is named df
  # Example DataFrame creation (for demonstration purposes)
  # Replace this with your actual DataFrame
  # df = pd.read_csv("your_file.csv")  # Read your actual file

  # Define the high-level index groups (e.g., 'Exposure_instate_DELTA_O3', 'Exposure_instate_DELTA_PM', etc.)
  high_level_indices = ['Exposure_national_DELTA_PM', 'Exposure_national_DELTA_O3',
                        'Exposure_instate_DELTA_PM', 'Exposure_instate_DELTA_O3']

  # Create a figure and subplots
  fig, axes = plt.subplots(nrows=len(high_level_indices), ncols=1, figsize=(14, 18))
  jj = 0
  pd.options.mode.chained_assignment = None
  # Iterate over the high-level index groups and corresponding axes
  for ax, high_level in zip(axes, high_level_indices):
      # Select the data for the current high-level index group
      data = df[high_level]

      for gi in calc_groups:
        data.loc[:, gi] = pt[high_level][gi]

      # Get the projects (index) and racial groups (columns)
      projects = data.index
      racial_groups = data.columns[0:7]

      # Plot each racial group as a separate bar in the same group (project)
      width = 0.1  # Width of each bar
      positions = np.arange(len(projects))  # Position of each group of bars
      for i, group in enumerate(racial_groups[:-1]):
          grp_absolute = group.replace("_relative", "")
          rects = ax.bar(positions + i * width, data[group]*100, width=width,
                         label=group)
          ax.bar_label(rects,
                       labels=[f'{-x:.2g}' if y > .10 else ""
                        for x, y in zip(data[grp_absolute], data[group])],
                       padding=3)

      # Set the title and labels
      ax.set_title(titles[jj], fontsize=14)
      ax.set_ylabel('% more or less exposed', fontsize=12)
      ax.set_xticks(positions + width * (len(racial_groups) / 2))
      if jj == 3:
        ax.set_xticklabels(projects, rotation=45, ha='right')
        ax.set_xlabel('Project', fontsize=16)
      else:
        ax.xaxis.set_ticklabels([])

      ax.grid(True, linestyle='--', alpha=0.7)
      jj += 1

  # Create a single legend for the entire figure at the bottom center
  handles, labels = ax.get_legend_handles_labels()
  labels = ['American Indian / Alaskan Native', 'Asian', 'Black and African American', 'Hispanic/Latino',
            'Native Hawaiian and Pacific Islander', 'Other']
  fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(racial_groups), title='Demographic Groups')

  # Adjust layout to prevent overlap
  plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust the layout to make room for the legend


  # Display the plot
  plt.show()

titles = ['PM2.5 exposure relative to White Americans', 'Ozone exposure relative to White Americans',
          'In-State PM2.5 exposure relative to In-State White Pop.',  'In-State Ozone exposure relative to In-State White Pop.']

In [ ]:
create_plots(pt2_planned, titles)